In [1]:
from utils_misc import pickle_load_or_create, pickle_save
from utils_cv2 import get_video_length, get_random_frame, get_squared_frames
from DataRepository import DataRepository
from DataGeneratorFrames import DataGeneratorRectangles
import numpy as np
import pandas as pd
import sys
import os

Defaulting to user installation because normal site-packages is not writeable


2024-08-04 14:06:50.419028: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-04 14:06:50.459688: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-04 14:06:51.220925: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
repo = DataRepository()

connection established


In [86]:
import numpy as np

def add_rectangle_to_frame(frame, rel_rect_center_x, rel_rect_center_y, rect_size):
    # Square, spatio-(temporal) variant
    height = frame.shape[0]
    width = frame.shape[1]
    size = max(width, height)
    offset_x = (size - width) // 2
    offset_y = (size - height)// 2

    print(offset_x, offset_y)
    # rect_center_x = x position in pixels within the frame (rectangle, not square)
    # Offset = the difference between height and width divided by 2

    # Xmin/max and Ymin/max in rectangle position
    rect_size_pixels = rect_size * size
    xmin = int(rel_rect_center_x * size - offset_x - (rect_size_pixels / 2))
    ymin = int(rel_rect_center_y * size - offset_y - (rect_size_pixels / 2))
    xmax = int(xmin + rect_size_pixels)
    ymax = int(ymin + rect_size_pixels)

    xmin = max(5, xmin)
    ymin = max(5, ymin)

    wanted_frame = np.array(frame[ymin:ymax, xmin:xmax])    
    frame[ymin-4:ymax+5, xmin-5:xmax+5] = [250,0,0] # y, x
    frame[ymin:ymax, xmin:xmax] = wanted_frame

    # Dot
    dotx = int(rel_rect_center_x * size - offset_x)
    doty = int(rel_rect_center_y * size - offset_y)
    frame[doty-10:doty+10, dotx-10:dotx+10] = [0,0,255]
    
    return frame

In [87]:
root = '/media/miked/Elements/Judge/FINISHED-DB-READY/'

In [119]:
id = 2
frameNr = 500

In [120]:
full_path = root + repo.get_path(id)
full_path

'/media/miked/Elements/Judge/FINISHED-DB-READY/free-vids/SR1/mike/mike-20240201-atelier-002.mp4'

In [121]:
frame = get_squared_frames(full_path, frameNr, frameNr, (600,600))[0]
frame.shape

(600, 600, 3)

In [122]:
lbls = repo.get_rectangles_from_batch(id, frameNr, frameNr).loc[0]
lbls

videoID            2.000000
frameNr          600.000000
label              2.000000
manual_insert      1.000000
rect_center_x      0.507292
rect_center_y      0.284896
rect_size          0.794801
Name: 0, dtype: float64

In [123]:
frame = add_rectangle_to_frame(frame, lbls['rect_center_x'], lbls['rect_center_y'], lbls['rect_size'])

In [124]:
import cv2
cv2.imshow('image',frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [101]:
repo.execute_command?

Signature: repo.execute_command(command)
Docstring: Execute function, to make sure it's always commited.
File:      ~/code/judge/scripts/DataRepository.py
Type:      method

In [38]:
qry = f"""
SELECT * FROM FrameLabels WHERE rect_center_x IS NOT NULL AND (rect_center_x * rect_center_y * rect_size != 0.25)
"""

In [40]:
lbls = repo.fetch_qry(qry).sort_values(['videoID', 'frameNr'])
lbls

,videoID,frameNr,label,manual_insert,rect_center_x,rect_center_y,rect_size
0,1,76,0,1,0.540741,0.188021,1.000000
1,1,77,0,1,0.540741,0.188021,1.000000
2,1,78,0,1,0.540741,0.188021,1.000000
3,1,79,0,1,0.540741,0.188021,1.000000
4,1,80,0,1,0.580556,0.181771,1.000000
...,...,...,...,...,...,...,...
36241,245,2005,0,1,0.454688,0.584259,0.642919
36242,245,2006,0,1,0.454688,0.584259,0.642919
36243,245,2007,0,1,0.454688,0.584259,0.642919
36244,245,2008,0,1,0.454688,0.584259,0.642919


In [12]:
# rect size delen door aspect ratio where not 0.5, 0.5, 1.0
# rel x, of rel y aanpassen afhankelijk van wat het kleinste is. dus based on videoID

In [72]:
for idx, lbl in lbls.iterrows():
    id = lbl['videoID']
    frameNr= lbl['frameNr']

    #cmd = f"UPDATE FrameLabels SET rect_center_x = {lbl['rect_center_x']}, rect_center_y = {lbl['rect_center_y']}, rect_size={lbl['rect_size']} WHERE videoID = {int(id)} AND frameNr = {int(frameNr)}"
    #repo.execute_command(cmd)

In [55]:
((0.8 * 1080) + 420) / 1920

0.66875

In [66]:
lbls.groupby('rect_size').count()

,videoID,frameNr,label,manual_insert,rect_center_x,rect_center_y
rect_size,,,,,,
0.176627,242,242,242,242,242,242
0.187384,21,21,21,21,21,21
0.187721,651,651,651,651,651,651
0.187891,124,124,124,124,124,124
0.193528,52,52,52,52,52,52
...,...,...,...,...,...,...
0.691802,2,2,2,2,2,2
0.725394,119,119,119,119,119,119
0.794801,450,450,450,450,450,450


In [13]:
repo.get_ageID('junsen')

5

In [14]:
gen = DataGeneratorRectangles(batch_size=8, train=True, rootfolder=root)

connection established
DataGeneratorSkillBorders init done
on_epoch_end_called


In [15]:
gen.batch_order.sort_values(['videoID','batch_nr_video'])


,videoID,batch_nr_video,frame_start,frame_end,frames,batch_id
1912,1,0,1,8,8,1912
3167,1,1,9,16,8,3167
1826,1,2,17,24,8,1826
3864,1,3,25,32,8,3864
1679,1,4,33,40,8,1679
...,...,...,...,...,...,...
766,245,247,1977,1984,8,766
1257,245,248,1985,1992,8,1257
96,245,249,1993,2000,8,96
618,245,250,2001,2008,8,618


In [16]:
gen.batch_order[(gen.batch_order['frames'] < 8)]


,videoID,batch_nr_video,frame_start,frame_end,frames,batch_id
18,2,104,833,833,1,18
48,110,118,945,947,3,48
128,2,178,1430,1432,3,128
216,3,186,1489,1489,1,216
341,30,101,816,816,1,341
374,52,211,1690,1696,7,374
520,110,233,1865,1870,6,520
616,2,179,1433,1435,3,616
762,5,317,2537,2538,2,762
826,30,249,1993,1994,2,826


In [17]:
gen.batch_order[(gen.batch_order['frames'] < 8) & (gen.batch_order['videoID'] == 52)]


,videoID,batch_nr_video,frame_start,frame_end,frames,batch_id
374,52,211,1690,1696,7,374
1833,52,0,1,3,3,1833
2319,52,210,1681,1687,7,2319
2676,52,254,2033,2034,2,2676
2763,52,1,14,16,3,2763


In [18]:
repo.get_rectangles_from_batch(100, 380,400)

,videoID,frameNr,label,manual_insert,rect_center_x,rect_center_y,rect_size
0,100,380,2,1,0.334375,0.605556,0.641185
1,100,381,2,1,0.334375,0.605556,0.680233
2,100,382,2,1,0.334375,0.605556,0.680233
3,100,383,2,1,0.316146,0.622222,0.700640
4,100,384,2,1,0.316146,0.622222,0.700640
5,100,385,2,1,0.316146,0.622222,0.721660
6,100,386,1,1,0.316146,0.622222,0.721660
7,100,387,1,1,0.316146,0.622222,0.788577
8,100,391,2,1,0.309896,0.624074,0.836601
9,100,392,2,1,0.309896,0.624074,0.836601


In [19]:
X, y = gen.__getitem__(2630)

In [20]:
X.shape

(8, 128, 128, 3)

In [21]:
y.shape

(8, 3)

In [22]:
y

array([[0.605556, 0.567187, 0.832972],
       [0.605556, 0.567187, 0.832972],
       [0.605556, 0.567187, 0.832972],
       [0.605556, 0.567187, 0.832972],
       [0.605556, 0.567187, 0.832972],
       [0.605556, 0.567187, 0.832972],
       [0.605556, 0.567187, 0.832972],
       [0.605556, 0.567187, 0.832972]])

In [23]:
repo.get_rectangles_from_batch(videoID=52, frame_start=14, frame_end=16)

,videoID,frameNr,label,manual_insert,rect_center_x,rect_center_y,rect_size
0,52,14,0,1,0.521354,0.586111,0.97
1,52,15,0,1,0.521354,0.586111,0.97
2,52,16,0,1,0.521354,0.586111,0.97


In [24]:
for i in gen.batch_order[(gen.batch_order['frames'] < 8)].index.values:
    #X, y = gen.__getitem__(i)
    #print(X.shape, y.shape)
    pass

In [25]:
model = pickle_load_or_create('../models/frames_skillborder_CNN_model_96pixels_history', lambda: None, True).model
print(model)

<Sequential name=sequential, built=True>


In [26]:
### Predicting frames

In [27]:
from utils_cv2 import get_squared_frames

In [28]:
vidid=222
frame_start=420
frame_end=580
path = root + repo.get_path(videoID=vidid)
path

'/media/miked/Elements/Judge/FINISHED-DB-READY/competition/belgium/DD4/bk-zeroskip-dd4-2024-junioren-yaro-j1.MP4'

In [29]:
X = get_squared_frames(path, frame_start, frame_end, dim=(256,256))
X.shape

(161, 256, 256, 3)

In [30]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 24)   │           672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 254, 254, 24)   │            96 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 252, 252, 32)   │         6,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 126, 126, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 124, 124, 48)   │        13,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 48)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 62, 62, 48)     │           192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 60, 60, 64)     │        27,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 30, 30, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 57600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     3,686,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,208,923 (42.76 MB)

 Trainable params: 3,736,195 (14.25 MB)

 Non-trainable params: 336 (1.31 KB)

 Optimizer params: 7,472,392 (28.50 MB)

In [31]:
X

array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        ...,

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]],


       [[[0., 0., 0.],
         [0., 0., 0.],
         [0., 

In [32]:
np.set_printoptions(suppress=True)
np.round(model.predict(X), 2)

6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 277ms/step


array([[ -99.98,  260.92,  318.24],
       [-100.21,  262.22,  320.56],
       [ -99.25,  260.82,  312.08],
       [ -98.75,  259.61,  307.75],
       [ -97.89,  258.22,  302.02],
       [ -98.33,  258.86,  301.85],
       [ -99.1 ,  259.98,  304.3 ],
       [ -99.52,  261.09,  307.94],
       [ -99.83,  261.03,  311.08],
       [-100.66,  262.26,  315.95],
       [-100.73,  261.58,  317.45],
       [-100.02,  260.2 ,  314.79],
       [-100.78,  262.11,  315.57],
       [-100.92,  262.59,  320.63],
       [-101.34,  263.37,  321.14],
       [-101.05,  262.48,  319.16],
       [-101.28,  262.5 ,  323.13],
       [-101.15,  262.57,  323.19],
       [-101.4 ,  262.9 ,  324.57],
       [-101.91,  263.89,  326.13],
       [-102.96,  265.82,  333.07],
       [-102.11,  264.4 ,  331.16],
       [-101.02,  261.92,  319.57],
       [-100.36,  259.77,  311.29],
       [-101.92,  262.93,  319.08],
       [-101.98,  264.07,  320.81],
       [-102.7 ,  265.25,  324.04],
       [-102.95,  265.09,  3

In [33]:
repo.get_rectangles_from_batch(vidid, frame_start, frame_end)

,videoID,frameNr,label,manual_insert,rect_center_x,rect_center_y,rect_size
0,222,420,1,1,0.551562,0.570833,0.527006
1,222,421,1,1,0.551562,0.570833,0.527006
2,222,422,1,1,0.551562,0.570833,0.527006
3,222,423,2,1,0.551562,0.570833,0.527006
4,222,424,2,1,0.551562,0.570833,0.527006
...,...,...,...,...,...,...,...
156,222,576,2,1,0.563281,0.583333,0.542816
157,222,577,2,1,0.563281,0.583333,0.542816
158,222,578,2,1,0.563281,0.583333,0.542816
159,222,579,2,1,0.563281,0.583333,0.542816
